In [ ]:
import pickle
import re
import pprint
import csv
import time
import pandas as pd
from collections import Counter
import pickle
import pandas as pd
import numpy as np
import re

In [ ]:
# get common data list
main_path = r'block_split_59_79.pickle'

with open(main_path, 'rb') as f:
    common_list = pickle.load(f)

In [ ]:
# ALL MAIN FUNCTIONS

# ______fixing mistakes with text blocks identification (the text block is attached after the header)________________
def drop_errors_with_blocks(common_list):
    new_common_list = []
    for el in common_list:
        path, block_list = el
        temp_list = []
        for str_block in block_list:
            searching = re.search(r'( +\-{5}> +\d+)|(\-{5}> +\d+)', str_block)
            if 'HEADER' in str_block:
                if searching:
                    corr_list = str_block.split('----->')
                    # print(corr_list)
                    # print(corr_list)
                    for temp_elem in corr_list:
                        pattern = re.match(r'( +\d+)', temp_elem)
                        pattern2 = re.search(r'[а-яё]', temp_elem)
                        if pattern:
                            temp_list.append(f'----->{temp_elem}')
                        elif 'HEADER' in temp_elem:
                            temp_list.append(f'----->{temp_elem}')
                        elif pattern2:
                            temp_list.append(f'----->{temp_elem}')
                        else:
                            pass
            else:
                temp_list.append(str_block)
        new_el = path, temp_list
        new_common_list.append(new_el)
    return new_common_list

# ____________find page numbers with additional markup_________________________
def get_pages(common_list):
    page_list = []
    log_list = []
    for elem in common_list:
        path, text_list = elem
        searching_text = text_list[-1]
        pattern = r'(PAGE +\d+)'
        try:
            row_page = re.findall(pattern, searching_text)  
            clean_page = row_page[-1].replace('PAGE', '').replace(' ', '')
            tuple_pages = path, clean_page
            page_list.append(tuple_pages)
        except Exception as ex:
            tuple_pages = path, 'NOT FOUND'
            page_list.append(tuple_pages)
    return page_list

# ________fixing mistakes with indexes___________________________
def pages_without_serias(data):
    serial_edition_list = []
    common_list = []

    pattern = 'серий'

    for el in data:
        img_path, text = el
        # print(type(img_path))
        matching = re.search(pattern, img_path)
        if matching:
            serial_edition_list.append(el)
        else:
            common_list.append(el)
    return common_list

def pages_without_ukaz(data):
    serial_edition_list = []
    common_list = []

    pattern = 'спомогательные'

    for el in data:
        img_path, text = el
        # print(type(img_path))
        matching = re.search(pattern, img_path)
        if matching:
            serial_edition_list.append(el)
        else:
            common_list.append(el)
    return common_list

# 'dvvu' indexes filtering
def get_dv_vu(some_list):
    bool_list = []
    for f in some_list:
        # if ',' in f:
        if 'ДВ ВУ' in f:
            # print(f)
            bool_list.append('False')
        else:
            bool_list.append('True')
    return bool_list

# filtering of errors with headers (check registration numbers in text blocks)
def filter_headers(common_list):
    pattern_reg = re.compile('(\d+-\d+[а-я])|(\d+-\d+)', 0)
    demo_2 = []
    for el in common_list:
        img_path, text = el
        new_text = []
        for block in range(len(text)):
            if 'HEADER' in text[block] and re.search(pattern_reg, text[block]):
                if block > 0:
                    new_text[-1] += text[block].replace('HEADER', ' ----->')
                else:
                    new_text.append(text[block].replace('HEADER', ' ----->'))
            else:
                new_text.append(text[block])
        el2 = img_path, new_text
        demo_2.append(el2)
    return demo_2


# get list with text block and filepaths
def get_row_uniqs(filt_header_common_list):
    uniq_preform_list = []
    for elem in filt_header_common_list:
        path, text_list = elem
        # new_text_list = []
        for t in text_list:
            pattern = re.compile(r'( +\-{5}> +\d+)|(\-{5}> +\d+)')
            matching = re.match(pattern, t)
            if 'HEADER' in t:
                if 'См.' in t:
                    # print(t)
                    t = t.replace('$$$', '')
                    useful_tuple = path, t
                    uniq_preform_list.append(useful_tuple)
                elif matching:
                    # print(t)
                    t = t.replace('$$$', '')
                    useful_tuple = path, t
                    uniq_preform_list.append(useful_tuple)
                else:
                    pass
            if 'HEADER' not in t:
                t = t.replace('$$$', '')
                useful_tuple = path, t
                uniq_preform_list.append(useful_tuple)
    return uniq_preform_list

# --------------fixing gaps --------------------
def formater(my_sample_list):
    temp_list = []
    for el in my_sample_list:
        el = list(el)
        str_list = []
        str_list.append(el[0])
        el[0]=str_list
        temp_list.append(el)
    return(temp_list)

def gap_detector(blocklist,idx):
    counter = 1
    text = blocklist[idx][1]
    compiler = re.compile(r'(\-{5}> +\d+)|( +\-{5}> +\d+)')
    pattern = re.match(compiler, text)
    if pattern:
        if idx+1< len(blocklist):
            idx=idx+1
            text = blocklist[idx][1]
            pattern = re.match(compiler, text)
            if not pattern:
                if not 'HEADER' in text:
                    trigger = detection = True
                    counter += 1
                    while trigger == True:
                        if idx+counter < len(blocklist):
                            next_text = blocklist[idx+counter-1][1]
                            next_pattern = re.match(compiler, next_text)
                            if not next_pattern:
                                if not 'HEADER' in next_text:
                                    trigger = detection = True
                                    counter += 1
                                else:
                                    counter = 1 #
                                    detection = False
                                    return detection, counter
                            else:
                                trigger = False
                                return detection, counter  
                        else:     
                            trigger = False  
                            return detection, counter
                else:
                    counter = 1 #
                    detection = False
                    return detection, counter
            else:
                counter = 1 #
                detection = False
                return detection, counter
        else:
            counter = 1 #
            detection = False
            return detection, counter
    else:
        counter =0 #
        detection = False
        return detection, counter
    
def recollector(blocklist,idx,lenght=1):
    *temp_my_adrr,temp_my_block = blocklist[idx]
    temp_my_adrr = temp_my_adrr[0]
    for idy in range(1,lenght):
        temp_my_block = f'{temp_my_block} {blocklist[idx+idy][1]}'
        temp_my_adrr.append(blocklist[idx+idy][0][0])
        temp_el = [temp_my_adrr, temp_my_block]
        blocklist[idx] = temp_el
    return blocklist

def collect_blocks_correction(collect_blocks):
    correction_uniqs = []
    for i in collect_blocks:
        temp_paths = []
        if len(i[0]) >= 1:
            uniq_i = list(set(i[0]))
            temp_paths.append(uniq_i)
        else:
            temp_paths.append(i)
        for a in temp_paths:
            a_paths = ', '.join(a)
        elem = [a_paths.split(','), i[1]]
        correction_uniqs.append(elem)
    return correction_uniqs

# fixing errors
# delete headers which begins with number
def check_false_headers(test_example_list):
    data1 = test_example_list.copy() # list copy

    for t in data1:
        pattern_dig = r'(\d+)'
        res = re.findall(pattern_dig, t[1])
        # print(res)
        if len(res) < 2:
            data1.remove(t)
    # print(data1)
    return data1

# # TEST___________FIND MULTIPLE BLOCKS_________________________________________
# def get_mult_num(test_string):
#     num_list = []
#     pattern = re.compile(r'(-{5}>  \d+-\d+ \. [А-ЯЁ])|(-{5}>  \d+-\d+\. [А-ЯЁ])')
#     matching = pattern.match(test_string, 0)
#     if matching:
#         ord_num = re.findall(r'(\d+-\d+)', test_string)
#         nums = ord_num[0].split('-')
#         num_list.append(nums)
#     else:
#         pass
#     return num_list

# del additional markup
def del_note_inf(text_list):
    new_text = []
    for t in text_list:
        tt = t.replace('PAGE', '').replace('NUM', '').replace('REG', '')
        new_text.append(tt)
    return new_text

def del_points(new_text):
    new_text_no_points = []
    for t in new_text:
        if t[0] and t[-1] == '"':
            tt = t.replace('"', '')
            new_text_no_points.append(tt)
        else:
            new_text_no_points.append(t)
    return new_text_no_points

# GET BLOCK TYPE
def get_block_kind(new_text_no_points):
    kind_block_list = []
    pattern_single = re.compile(r'(-{5}>  \d+\.)')
    pattern_mult = re.compile(r'(-{5}>  \d+-\d+\.)|(-{5}>  \d+-\d+ \.)')
    for n in new_text_no_points:
        sing_match = pattern_single.match(n, 0)
        mult_match = pattern_mult.match(n, 0)
        if sing_match:
            str_sing = 'single'
            kind_block_list.append(str_sing)
        elif mult_match:
            str_mult = 'multiple'
            kind_block_list.append(str_mult)
        else:
            str_error = 'error'
            kind_block_list.append(str_error)
    return kind_block_list


# GET MULT BLOCKS
def get_mult_num(test_string):
    num_list = []
    ord_num = re.findall(r'(\d+-\d+)', test_string)
    nums = ord_num[0].split('-')
    num_list.append(nums)
    return num_list

#get series of ordinal numbers
def get_list_ords(nums):
    first_str = nums[0][0] #string
    sec_str = nums[0][1]
    # first_str_idx = len(nums[0][0]) #string length
    sec_str_idx = len(nums[0][1])
    srez = first_str[:-sec_str_idx]
    sec_num_str = srez + sec_str

    ords_list = list(range(int(first_str), int(sec_num_str)+1))
    return ords_list

# function for sorting unique text blocks 
def get_uniqs(test_string, ords_list, uniq_list_fin = []):
    multiple_block_str = ''
    multiple_block_str += test_string
    multiple_block_str += ' '
    arrow = re.findall(r'-{5}>', multiple_block_str)
    parts_block = multiple_block_str.split('-----> ') #the first element is empty
    uniq_list = []
    for part in parts_block:
        # print(part)
        # uniq_list = []
        if len(arrow) - 1 == len(ords_list):
            # print("YES")
            main_part = parts_block[1]
            uniq = (f'-----> {main_part} {part}')
            # print(uniq)
            uniq_list.append(uniq)
            # print(uniq_list)
            uniq_list_fin = [(n, st) for n, st in zip(ords_list, uniq_list[2:])]

    return uniq_list_fin

# filtering 'OBU' indexes
def get_obu(some_list):
    bool_list = []
    for f in some_list:
        # if ',' in f:
        if 'спомогательные' in f:
            # print(f)
            bool_list.append('False')
        elif '/ОБУ/' in f:
            # print(f)
            bool_list.append('False')
        else:
            bool_list.append('True')
    return bool_list

# get main elements in image paths 
# sample = ['./letopisi/1979/№9/0466.jpg', 
#               './letopisi/1969/дополнительный выпуск 3/0019.jpg', 
#               './letopisi/1969/дополнительный выпуск 3/0020.jpg']
def get_main_cut_paths(sample_two):
    pattern_two = r'(\/19.+\.jpg)'
    main_cut_paths = []
    for s in sample_two:
        matching_two = re.findall(pattern_two, s)

        main_cut_list_paths = []

        for match in matching_two:
            cut_path_two = match.replace('№', '')
            # print(cut_path)
            main_cut_list_paths.append(cut_path_two[1:])
        
        main_cut_paths.append(main_cut_list_paths)
    return main_cut_paths

# get edition year values
def get_year(some_list):
    list_year = []
    pattern_year = r'(19\d+)'
    # find = re.findall()
    for i in some_list:
        for year in i:
            find = re.findall(pattern_year, year)
            list_year.append(find[0])
    return list_year

# get edition number
error_list = []
def get_edition(path_list):
    my_edition = []
    some_eds = []
    pattern = r'(\d+)'
    for p in path_list:
        temp_list = p.split('/')
        elem_ed = temp_list[-2]
        edition = re.search(pattern, elem_ed)
        if edition:
            some_eds.append(edition.group(0))
        else:
            # print(p)
            error_list.append(p)
    if len(some_eds) >= 1:
        main_numb = some_eds[0]
        my_edition.append(main_numb)
    else:
        error_list.append(p)
    a = ''.join(my_edition)
    if a.isdigit():
        return int(a)

# get number of jpeg from image paths
error_jpg_list = []
def get_jpg_num(path_list):
    some_jpg = []
    pattern = r'(\d+\.jpg)'
    for p in path_list:
        temp_list = p.split('/')
        elem_ed = temp_list[-1]
        edition = re.search(pattern, elem_ed)
        if edition:
            some_jpg.append(edition.group(0).replace('.jpg', ''))
        else:
            # print(p)
            error_jpg_list.append(p)
    return list(some_jpg)

In [ ]:
page_list = get_pages(common_list) #get page number from every scan

# delete auxiliary indexes and serial indexes + erroneous headers
common_list_2 = pages_without_serias(page_list)
common_list_3 = pages_without_ukaz(common_list_2)
filt_header_common_list = filter_headers(common_list_3)
# unique text blocks list
uniq_preform_list = get_row_uniqs(filt_header_common_list)

In [ ]:
# MAIN CIRCLE
# fixing of gaps in text blocks in cases where text block is on page breaks and get unique text blocks
start_time = time.time()
poplist=[]
collect_blocks=[]
try:       
    t_list = formater(uniq_preform_list)
    for idx, block in enumerate(t_list):
    # print(idx, block)
        *isgap,lenght = gap_detector(t_list,idx)
    # print(isgap, lenght)
        if isgap[0]:
            collect_blocks = recollector(t_list,idx, lenght)
    # print(collect_blocks)
        if isgap[0] == False and lenght==0: 
            poplist.append(idx)
    poplist = sorted(poplist, reverse=True)
    # print(poplist)
    for el in poplist:
        collect_blocks.pop(el)
except Exception as e:
    with open('logs_gaps.log', 'a', encoding = 'utf8') as f:
        f.write(f'{e}')
#         pass
end_time = time.time()

circle_working_time = end_time - start_time
print(circle_working_time)

In [ ]:
# get row data with unique text blocks
test = collect_blocks_correction(collect_blocks)
# delete headers which begins with figures
data1 = check_false_headers(test)

# get dataframe with paths and text blocks
df = pd.DataFrame(data1, columns = ['paths', 'blocktext'])
df.paths = df.paths.apply(tuple) #conversion to common type of data
# create the second dataframe with pages on scans
page_df = pd.DataFrame(page_list, columns = ['paths', 'num_of_page'])

# merger of dataframe where every text block linked with scan of Book Chronicles
df = df.reset_index(names = 'block_id')
df= df.explode('paths')
df = df.reset_index(drop = True)
df.paths = df.paths.str.strip()
merged_df = df.merge(page_df, how = 'outer')

In [ ]:
# correction of dataframe formating
merged_df.paths = merged_df.paths.apply(lambda x: x +', ')
merged_df.num_of_page = merged_df.num_of_page.apply(lambda x: x +', ')
new_df = merged_df.groupby('block_id').agg({'paths':'sum','num_of_page':'sum','blocktext':'first'}).reset_index()
new_df.num_of_page = new_df.num_of_page.str.strip(', ')
new_df.paths = new_df.paths.str.strip(', ')

In [ ]:
# do text correction
text_list = new_df['blocktext'].to_list()

# MAIN FOR FINAL COMMON DATAFRAME _______________________________________________________
no_note = del_note_inf(text_list) # delete words from additional markup
no_points = del_points(no_note) # delete quotation marks
list_block_kinds = get_block_kind(no_points) # get block type to list
# circle for assembly of multiple text blocks
final_block_list = []
for one_text, kind_block in zip(no_points, list_block_kinds):
    if kind_block == 'multiple':
        nums = get_mult_num(one_text)
        order_nums = get_list_ords(nums)
        uniqs_from_multi = get_uniqs(one_text, order_nums)
        final_block_list.append(uniqs_from_multi)
    else:
        final_block_list.append(one_text)

In [ ]:
# добавочный датафрейм сливаем с основным
add_dict = {'block_kind': list_block_kinds, 'final_blocks': final_block_list}

add_df = pd.DataFrame(add_dict)

final_df = new_df.join(add_df)

In [ ]:
# working with dataframe and filtering paths
first = final_df['paths'].tolist()
bool_first = get_obu(first)
final_df['not_obu'] = bool_first
main_df = final_df.loc[final_df['not_obu'] != 'False']
main_df = main_df.reset_index()
del main_df['block_id']

sample_two = main_df['paths'].tolist()
main_cut_paths = get_main_cut_paths(sample_two)
main_df['cut_paths'] = main_cut_paths
del main_df['index']
del main_df['level_0']
del main_df['not_obu']
del main_df['block_id']

In [ ]:
# get years of issue
list_for_years = main_df['cut_paths'].tolist()
sample_year = get_year(list_for_years)
main_df['year'] = sample_year

In [ ]:
# get editions from image paths
main_df['num_edition'] = main_df['cut_paths'].apply(get_edition)
# print(error_list)

In [ ]:
# get jpeg-file number of image
main_df['jpg_num'] = main_df['cut_paths'].apply(get_jpg_num)

In [ ]:
# # TEST
# # check pass values
# print(set(main_df['num_edition'].isnull()))

In [ ]:
# sorting of dataframe by chronology
d = main_df[main_df.isna().sum(axis=1).eq(1)]
df_clear = main_df.dropna().reset_index()
sort_df = df_clear.sort_values(['year', 'num_edition'])
sort_csv_df = sort_df.reset_index()
del sort_csv_df['level_0']
del sort_csv_df['edition']

In [ ]:
# filtering 'dvvu' pointers in paths
add_bool_list = sort_csv_df['paths'].tolist()
res = get_dv_vu(add_bool_list)
sort_csv_df['bool_dvvu'] = res
new_df = sort_csv_df.drop(sort_csv_df.loc[sort_csv_df['bool_dvvu'] == 'False'].index)
new_df = new_df.reset_index()
del new_df['index']
del new_df['bool_dvvu']
del new_df['level_0']
new_df['jpgs'] = new_df['cut_paths'].apply(get_jpg_num)
# save result
# new_df.to_csv('sort_df_all.csv', sep = ';', encoding='utf8')

In [ ]:
# # TEST
# # when you need only single text blocks
# only_single = new_df.loc[new_df['block_kind'] == 'single']
# del only_single['jpgs']
# only_single.reset_index()
# only_single.shape

# # when you need only multiple text blocks
# only_multiple = new_df.loc[new_df['block_kind'] == 'multiple']
# multiple_uniqs = only_multiple.loc[only_multiple['final_blocks'] != '[]']

In [ ]:
# # TEST
# # check and count of multiple text blocks in dataframe

# checking = final_df['final_blocks'].to_list()
# count_mult_blocks = []
# for block in checking:
#     try:
#         if type(block) == list:
#             # print(block)
#             len_block = len(block)
#             count_mult_blocks.append(len_block)
#     except:
#         pass


In [ ]:
# # TEST
# # STATISTICS__________________
# # count of unique text blocks in multiple blocks
# nul_blocks = set(count_mult_blocks)
# # added multiple blocks
# sum_mult_blocks = sum(count_mult_blocks)
# # count multiple blocks (value 0)
# c = Counter(count_mult_blocks)
# # count different text block's types
# checking_error = final_df['block_kind'].to_list()
# c_error = Counter(checking_error)
# print(c_error)
# print(c)